# Criação dos Exemplos

In [1]:
from mgd import pipelines
from itertools import chain
import pandas as pd

In [2]:
mapper = {
    'blues': 0,
    'classical': 1,
    'country': 2,
    'disco': 3,
    'hiphop': 4,
    'jazz': 5,
    'metal': 6,
    'pop': 7,
    'reggae': 8,
    'rock': 9
}


In [4]:
for s in ['scenario3']:
    print(f'Started {s}')

    examples = []
    directories = list(mapper.keys())

    for d in directories:
        examples.append(pipelines.extract_examples_from_dir(
            f'../gtzan/raw/{d}', norm_agg=s))

    examples = list(chain.from_iterable(examples))
    data = {
        'song_id': [],
        'label': [],
        'spectral_centroid': [],
        'spectral_flatness': [],
        'spectral_roll_of': []
    }
    mfcc_dict = {f'mfcc_{i}': [] for i in range(1, 13)}
    tonnetz_dict = {f'tonnetz_{i}': [] for i in range(1, 7)}
    data = data | mfcc_dict | tonnetz_dict

    for e in examples:
        data['song_id'].append(e.song_id)
        data['label'].append(mapper[e.label])
        data['spectral_centroid'].append(e.features.sc.item(0))
        data['spectral_flatness'].append(e.features.sf.item(0))
        data['spectral_roll_of'].append(e.features.sr.item(0))

        for i in range(1, 13):
            mffc_str = f'mfcc_{i}'
            data[mffc_str].append(e.features.mfcc.item(i))

        for i in range(1, 7):
            tonnetz_str = f'tonnetz_{i}'
            data[tonnetz_str].append(e.features.tonnetz.item(i-1))

    print(f'Finished {s}')
    examples_df = pd.DataFrame(data)
    examples_df.to_csv(f'../gtzan/processed/mgd_{s}.csv', index=False)


Started scenario3
